In [17]:
import cv2
import mediapipe as mp
import time
import numpy as np
import math
import pycaw
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [18]:

class handDetector():
    def __init__(self,mode = False,maxHands = 2,DetectionCOn = 0.5, trackcon = 0.5):
        #initialising variable for Hands thought not used here 
        self.mode = mode
        self.maxHands = maxHands
        self.DetectionCOn = DetectionCOn
        self.trackcon = trackcon
        
        #using hands funtion to get the hand detected
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands()
        self.mpDraw = mp.solutions.drawing_utils

    #drawing basic hand  
    def findHands(self, img,draw = True):
        self.imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(self.imgRGB)
        #print(results.multi_hand_landmarks)   #will print co-ordinatates of hand


        #Getting the image and masking the nodes on it
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS)
        return img


    #Add this as well below for loop if you want edge by edge
    def findPosition(self, img, handNo = 0, draw = True):
        lmlist= []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id,lm in enumerate(myHand.landmark):
                            h,w,c = img.shape
                            cx,cy = int(lm.x*w) , int(lm.y*h)
                            lmlist.append([id,cx,cy])  #this will give the point and the coordinate of the point
                            if  draw:   #Checking which id is where and creating a circle around it
                                cv2.circle(img,(cx,cy),15,(255,0,0),cv2.FILLED)    
        return lmlist





In [27]:

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(-20.0, None)
minVol = volRange[0]
maxVol = volRange[1]
print(minVol,maxVol)

-63.5 0.0


In [35]:
cap = cv2.VideoCapture(0)
detector = handDetector()
pTime = 0
vol = 0
volbar = 400
volper = 0
while True:
    success , img = cap.read()
    detector.findHands(img, draw = False)
    lmlist =  detector.findPosition(img, draw = False)
    if len(lmlist) !=0:
        
        #circling the index and thumb as well as drawing a line between them
        x1,y1 = lmlist[4][1] ,lmlist[4][2]
        x2,y2 = lmlist[8][1] ,lmlist[8][2]
        cx,cy =  (x1 + x2) //2 ,(y1 +y2) //2
        cv2.circle(img,(x1,y1),5,(255,0,255),cv2.FILLED)
        cv2.circle(img,(x2,y2),5,(255,0,255),cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
        cv2.circle(img,(cx,cy),5,(255,100,0),cv2.FILLED)
        
        lenght = math.hypot(x2-x1, y2-y1)
        
        
        #Volume Changer code
        vol = np.interp(lenght,[30,200],[minVol,maxVol])
        volbar = np.interp(lenght,[30,200],[400,150])
        volper = np.interp(lenght,[30,200],[0,100])
        
        
        print(lenght)
        volume.SetMasterVolumeLevel(vol, None)
        if lenght<50:
            cv2.circle(img,(cx,cy),5,(0,255,0),cv2.FILLED)
            
            
            
    cv2.rectangle(img,(50,150),(85,400),(0,255,0),3)
    cv2.rectangle(img,(50,int(volbar)),(85,400),(0,255,0),cv2.FILLED)
    cv2.putText(img,str(int(volper)),(40,450),cv2.FONT_HERSHEY_COMPLEX,2,(250,30,32),1)        
            
            
    #fps module
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_COMPLEX,2,(250,30,32),1)
    cv2.imshow("image",img) 
    
    
    if cv2.waitKey(1) == ord('q'):
        break 

164.5600194457937
178.84630272946657
183.84776310850236
197.03045449879062
198.59506539690256
199.86495440671933
203.9730374338726
203.06649157357302
198.76116320851014
156.52475842498527
97.29337079164233
82.0
76.68767828015137
70.61161377563892
70.83078426785913
68.65857557508748
69.12307863514182
65.73431371817918
64.32728814430156
67.468511173732
85.37564055396598
101.24228365658294
123.97580409095961
123.9395013706284
150.81445554057476
182.64993840677855
194.03092537015846
204.78769494283586
212.13203435596427
228.39658491317246
227.98245546532743
228.37250272307304
227.27076362788065
221.0
220.10224896624752
202.09403751719148
176.3774361986249
151.6047492659778
141.8767070381886
142.04224723651762
118.66338946785567
111.00450441310929
111.19802156513397
105.90561835898981
105.11898020814318
104.1393297462587
103.94229168149026
104.1393297462587
101.78899744078434
107.35455276791944
105.38026380684383
112.8716084761797
111.87939935484101
119.06720791217035
120.06664815842908
118

Docstring:
putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]]) -> img
.   @brief Draws a text string.
.   
.   The function cv::putText renders the specified text string in the image. Symbols that cannot be rendered
.   using the specified font are replaced by question marks. See #getTextSize for a text rendering code
.   example.
.   
.   @param img Image.
.   @param text Text string to be drawn.
.   @param org Bottom-left corner of the text string in the image.
.   @param fontFace Font type, see #HersheyFonts.
.   @param fontScale Font scale factor that is multiplied by the font-specific base size.
.   @param color Text color.
.   @param thickness Thickness of the lines used to draw a text.
.   @param lineType Line type. See #LineTypes
.   @param bottomLeftOrigin When true, the image data origin is at the bottom-left corner. Otherwise,
.   it is at the top-left corner.
Type:      builtin_function_or_method